---
### Chapter 2: Trajectory Optimisation
---
# **Tutorial 2.b - Contacts**
**Aim:** To finally make a friggin' leg! Aw yisss. But also to properly confront the problem of hard contacts, and what we do about them.

**Further reading:** There are two important papers I'll be referring to, but I'd suggest rather reading them afterwards:
1. <a href="https://jeb.biologists.org/content/jexbio/202/23/3325.full.pdf"> Templates and Anchors: Neuromechanical Hypotheses of Legged Locomotion on Land - RJ Full and DE Koditschek (1999) </a>
2. <a href=https://journals.sagepub.com/doi/pdf/10.1177/0278364913506757> A direct method for trajectory optimization of rigid bodies through contact - M Posa, C Cantu and R Tedrake (2014) </a>

If you have a bit more time and want to immerse yourself in one of the religious texts of legged robotics, check out *Legged Robots that Balance* by Marc Raibert (the dude founded Boston Dynamics, so I reckon he knows what he's talking about.)

### **Contents**:
* [](#)

## **Templates and Anchors**
The model we'll be constructing consists of one leg (two links connected by a prismatic joint) and a body:

<img src = "hopper.png" width = "400">

A monopedal hopper probably isn't the most useful configuration in real life, but it's still an important model that helps biologists and roboticists alike to understand legged locomotion.

Considering that most of the macroscopic, multicellular life forms on Earth are insects, the average number of legs that an animal has is... in my estimation... more than one. So you may be wondering how a one-legged model can tell us anything useful about any real critter. But imagine somebody told you to come up with the shortest possible "executive summary" of the running motion of, say, a cheetah. (It's always a cheetah... ;P) The first thing you'd do is remove all the non-essential bits: so head, flexy torso and tail become just one rigid body. Or maybe even a point mass. And then you notice that only one leg is ever really in contact with the ground at a time, and hey, even if more than one was in contact, there is only one centre of pressure, so why do we even need more than one 'leg?\*'... And soon enough, you're left with a hopper. The outcome would be the same if you tried it for a human, or a cockroach, or pretty much any other legged animal you could think of (except maybe a millipede... I suspect locomotion fundamentals change when the number of legs hits double figures...) 

\* this single "leg" (well, limb betweent the body and centre of pressure) used to represent the action of more than one leg is referred to as a *virtual leg.*

In biomechanics, this "executive summary" model for a motion - i.e. the lowest degree-of-freedom model that still captures its essential characteristics - is called a *template*. More complex models are called *anchors*, since they anchor the template behaviour into the more realistic, detailed behaviour of the system. In addition to the obvious simplification benefit, templates are beneficial because they allow us to compare the motion of systems with widely differing morphologies.  

...So that's why we care about monopeds: from Raibert's bouncing robots to the spring-loaded inverted pendulum (SLIP) models frequently applied to represent running, it's clear they are a good template for legged locomotion. 

**The takeaway:** Rather than just giving you some cool biomechanical jargon to throw around, the thing I want you to get out of this template idea is that dynamic modelling should be a modular process. Trajectory optimization is extremely computationally demanding, so we have to be as economical as possible with our degrees of freedom. Being efficient with your coordinates can make the difference between finding a solution in 15 minutes, or three hours.

Of course, that doesn't mean we should never use high-order models: anchors are absolutely essential to understanding motion. It just means you should get the simplest model working first, and then build complexity slowly and iteratively from there.

## **Hopper Equations of Motion**
Let's quickly get the dynamics of the hopper out of the way. We'll use 5 generalized coordinates: the position and angle of the body, the hip angle, and the length of the prismatic joint at the knee. We'll add actuator forces at both joints, and ground reaction forces at the foot. The activation of these forces will be controlled with complementarity constraints.

In [ ]:
# SYMBOLIC MODELLING CODE

# import libraries
import sympy as sym
import numpy as np

from IPython.display import display, HTML #for pretty printing
display(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 32em; }</style>"))

# create symbolic variables

# system parameters
g = sym.symbols('g')
m1 ,  m2,  m3 = sym.symbols([ 'm_{body}', 'm_{thigh}', 'm_{shank}']) # mass of links
l1 ,  l2,  l3 = sym.symbols([ 'l_{body}', 'l_{thigh}', 'l_{shank}']) # length of links
d1 ,  d2,  d3 = sym.symbols([ 'd_{body}', 'd_{thigh}', 'd_{shank}']) # length of links
In1, In2, In3 = sym.symbols(['In_{body}','In_{thigh}','In_{shank}']) # moment of intertia of links

# generalized coordinates

# body x position, body y position, body angle, leg angle, prismatic joint extension
x  ,   dx,   ddx = sym.symbols(['x'             ,'\dot{x}'             ,'\ddot{x}']) # note that x and y are not fixed as they were in prev tutorials
y  ,   dy,   ddy = sym.symbols(['y'             ,'\dot{y}'             ,'\ddot{y}'])
th1, dth1, ddth1 = sym.symbols(['\\theta_{body}','\dot{\\theta_{body}}','\ddot{\\theta_{body}}'])
th2, dth2, ddth2 = sym.symbols(['\\theta_{leg}' ,'\dot{\\theta_{leg}}' ,'\ddot{\\theta_{leg}}'])
s  ,   ds,   dds = sym.symbols(['s'             ,'\dot{s}'             ,'\ddot{s}'])

q   = sym.Matrix([  [x],  [y],  [th1],  [th2],  [s]]) #group into matrices
dq  = sym.Matrix([ [dx], [dy], [dth1], [dth2], [ds]])
ddq = sym.Matrix([[ddx],[ddy],[ddth1],[ddth2],[dds]])

#--------------------------------------------------------------------------------------------------

# STEP 1: system space coordinates written in terms of the generalised coordinates

# helper functions
def Rotate(v, th):
    # the 2D system space coordinates are [x;y;th], so we need a rotation function that can work with this
    R = sym.Matrix([[sym.cos(th), -sym.sin(th), 0],
                    [sym.sin(th),  sym.cos(th), 0],
                    [          0,            0, 1]]) # rotation matrix, augmented because of the theta element of the vector
    S = sym.Matrix([[0],[0],[th]]) # angle of rotation
    return R*v + S # coordinates after rotation

def GetXY(v):
    # this function is for applying a position-only offset (preserves the angle)
    vector_mask = sym.Matrix([[1],[1],[0]]) # gets rid of angle component of vector when multiplied elementwise
    return v.multiply_elementwise(vector_mask)

# positions of each link in their own reference frames
r0   = sym.Matrix([[x],     [y],[0]])
r1_1 = sym.Matrix([[0],     [0],[0]]) # writing it out just for consistency
r2_2 = sym.Matrix([[0],[-d2*l2],[0]])
r3_3 = sym.Matrix([[0],[-d3*l3],[0]])
rs_2 = sym.Matrix([[0],    [-s],[0]])

# Absolute orientations

# positions of each link, moved into the inertial frame
r1_0 = Rotate(r1_1, th1) + GetXY(r0)

r2_1 = Rotate(r2_2, th2-th1) + GetXY(r1_1/d1*(1-d1))
r2_0 = Rotate(r2_1, th1) + GetXY(r1_0)

r3_2 = r3_3 + rs_2 + GetXY(r2_2/d2*(1-d2)) # in order: position of link 3 COM in frame 3, length of prismatic joint, bottom portion of link 2
r3_0 = Rotate(r3_2, th2) + GetXY(r2_0)

# joint and end positions. NOT STRICTLY NECESSARY, but makes plotting stuff easier.
rBodyL_0 = Rotate(sym.Matrix([[   -d1*l1],[0],[0]]), th1) + GetXY(r1_0)
rBodyR_0 = Rotate(sym.Matrix([[(1-d1)*l1],[0],[0]]), th1) + GetXY(r1_0)

rKnee1_0 = Rotate(r2_2/d2*(1-d2)       , th2) + GetXY(r2_0) # top of knee
rKnee2_0 = Rotate(r2_2/d2*(1-d2) + rs_2, th2) + GetXY(r2_0) # bottom of knee (remember: prismatic joint)

# Foot position -> necessary for ground contact forces
rFoot_0 = Rotate(r3_3/d3*(1-d3), th2) + GetXY(r3_0)

#--------------------------------------------------------------------------------------------------

# STEP 2: generate expressions for the system space velocities

dr1 = r1_0.jacobian(q)*dq
dr2 = r2_0.jacobian(q)*dq
dr3 = r3_0.jacobian(q)*dq

drFoot = rFoot_0.jacobian(q)*dq

#--------------------------------------------------------------------------------------------------

# STEP 3: generate expressions for the kinetic and potential energy

# helper functions
def Ek(m, In, dr):
    InM = sym.Matrix([[m,0,0],[0,m,0],[0,0,In]])
    return 0.5*dr.T*InM*dr

def Ep(m, r):
    return sym.Matrix([m*g*r[1]])

# expressions
T = Ek(m1, In1, dr1) + Ek(m2, In2, dr2) + Ek(m3, In3, dr3)
V = Ep(m1, r1_0) + Ep(m2, r2_0) + Ep(m3, r3_0)

#--------------------------------------------------------------------------------------------------

# STEP 4: calculate each term of the Lagrange equation

# term 1
Lg1 = sym.zeros(1,len(q))
for i in range(len(q)):
    dT_ddq = sym.diff(T,dq[i]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 2 = 0
# term 3
Lg3 = T.jacobian(q) # partial of T in q

# term 4
Lg4 = V.jacobian(q) # partial of U in q

#--------------------------------------------------------------------------------------------------

# STEP 5: calculate generalized forces

tau, f, GRFx, GRFy = sym.symbols(['tau', 'F', 'GRF_{x}', 'GRF_{y}']) # arbitrary control torque
tau_l1 = sym.Matrix([[0],[0],[-tau]]) # the world frame torque at the hip, acting on the body
tau_l2 = sym.Matrix([[0],[0],[ tau]]) # the world frame torque at the hip, acting on the thigh
f_l2   = Rotate(sym.Matrix([[0],[ f],[0]]), th2) # the world frame force at the knee, acting on the thigh
f_l3   = Rotate(sym.Matrix([[0],[-f],[0]]), th2) # the world frame force at the knee, acting on the shank
GRF_l3 = sym.Matrix([[GRFx],[GRFy],[0]])

Qtau = r1_0.jacobian(q).T*tau_l1 + r2_0.jacobian(q).T*tau_l2
Qf = r2_0.jacobian(q).T*f_l2 + r3_0.jacobian(q).T*f_l3
QGRF = rFoot_0.jacobian(q).T*GRF_l3

Qtot = Qtau + Qf + QGRF
# -----------------------------------------------------------------------------------

# STEP 6: put it all together

EOM = Lg1 - Lg3 + Lg4 - Qtot.T
EOM = EOM.T

display(EOM[1].simplify())

In [ ]:
# import libraries
# Pyomo stuff
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

#Lambdify the EOM

func_map = {'sin':sin, 'cos':cos} 
# You need to tell 'lambdify' which symbolic toolbox functions = which functions from other modules.
# Here, we want the symbolic sin and cos to map to pyomo's sin and cos.
# (Yes, pyomo has its own trig functions that are distinct from numpy's or math's. You need to use them.)

sym_list = [g,
            m1, m2, m3,
            l1, l2, l3,
            d1, d2, d3,
            In1, In2, In3,
            x  ,  y, th1,   th2,  s,
            dx , dy, dth1, dth2, ds,
            ddx,ddy,ddth1,ddth2,dds,
            tau, f, GRFx, GRFy] # list of the symbols that will be substituted with inputs


DOFs  = ['x','y','th1','th2','s']

lamb_EOM  = {}
for dof_i, dof in enumerate(DOFs):
    lamb_EOM[dof] = sym.lambdify(sym_list, EOM[dof_i],modules = [func_map])

### **Pro tip: lambdify your aux variable definitions**
As you add links to the leg, the expressions for the positions, and especially the velocities of the contacts get very unweildly very quickly. Luckily, the LORD hath given unto us sympy, that we need fear no kerfuffle. I lambdify the expressions for the positions and velocities of the contact points to make my life easier when I construct the relevant constraints.

In [ ]:
# Lambdify your aux variables -> makes life easier later

TDOFs = ['x','y']
plotpoints = ['bodyL', 'bodyR', 'hip', 'knee1', 'knee2', 'foot']
lamb_footp = {}
lamb_footv = {}
lamb_plotp = {}

for dof_i, dof in enumerate(TDOFs):
    lamb_footp[dof] = sym.lambdify(sym_list,rFoot_0[dof_i],modules = [func_map])
    lamb_footv[dof] = sym.lambdify(sym_list, drFoot[dof_i],modules = [func_map])

    lamb_plotp.update({('bodyL', dof) : sym.lambdify(sym_list, rBodyL_0[dof_i],modules = [func_map])})
    lamb_plotp.update({('bodyR', dof) : sym.lambdify(sym_list, rBodyR_0[dof_i],modules = [func_map])})
    lamb_plotp.update({('hip'  , dof) : sym.lambdify(sym_list,     r1_0[dof_i],modules = [func_map])})
    lamb_plotp.update({('knee1', dof) : sym.lambdify(sym_list, rKnee1_0[dof_i],modules = [func_map])})
    lamb_plotp.update({('knee2', dof) : sym.lambdify(sym_list, rKnee2_0[dof_i],modules = [func_map])})
    lamb_plotp.update({('foot' , dof) : sym.lambdify(sym_list,  rFoot_0[dof_i],modules = [func_map])})

## **The Pyomo Model**

In [ ]:
# create the model
m = ConcreteModel()

# SETS-----------------------------------------------------------------------------------------------------------------------

N = 100 # for sprint task
# N = 20 # for sanity checks
m.N = RangeSet(N)

# sets can have multidimensional indices
# (probably a little gratuitous for such a simple model, but thought I'd show you anyway)
links = [('body',1),('leg',1),('leg',2)]
m.L = Set(dimen = 2, initialize = links)
m.pp = Set(initialize = plotpoints)

m.DOF = Set(initialize = DOFs) # the coordinates for each link
m.TDOF = Set(initialize = TDOFs) # world-frame coordinates for contact variables

***Annoying sidenote on the subject of set creation**

When you create a set, always do it like this:

```python
SpiceGirls = ['ginger','baby','posh','scary','sporty']
m.sg = Set(initialize = SpiceGirls)
```
or perhaps like this:

```python
m.sg = Set(initialize = ['ginger','baby','posh','scary','sporty'])
```
(though the first one is nicer, because having a list of the names it makes it easier to iterate over members of that set later if you have to.)

But **never** like this:

```python
m.sg = Set(['ginger','baby','posh','scary','sporty'])
```
Daft as it is, that last one actually creates five empty *sets*, instead of what you want: a single set with five values.

You can probably get away with it in many circumstances, but attempting to index using a set obviously makes absolutely no sense, so you're going to run into an error sooner or later. (If you ever come across the error **"cannot index a component with an indexed set"**, this may be why.

In [ ]:
# PARAMETERS-----------------------------------------------------------------------------------------------------------------

m.g = Param(initialize = 9.81)
# whenever a multidimensional set is passed to a function, the index is expanded (infuriatingly enough...)
# so every function indexed on L needs to expect two inputs from that set 
# I've called them l = [lb, ln] for 'branch' and 'number'

def get_m(n, lb, ln):
    if lb == 'body':
        return 0.5
    else: return 0.25
# note that the masses add up to 1
m.m = Param(m.L, initialize = get_m) # mass of links

def get_len(n, lb, ln):
    if lb == 'body':
        return 1.0
    else: return 0.5
m.len = Param(m.L, initialize = get_len) # length of links

m.dCOM = Param(m.L, initialize = 0.5) # distance from top of link to COM

def calculate_In(m, lb, ln): 
    l = (lb,ln)
    # yes, that does mean you have to rebuild the tuple inside the function.
    return m.m[l]*m.len[l]**2/12 
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

# VARIABLES -----------------------------------------------------------------------------------------------------------------

# system coordinates
m.q   = Var(m.N, m.DOF) # position
m.dq  = Var(m.N, m.DOF) # velocity
m.ddq = Var(m.N, m.DOF) # acceleration

# joint forces
joints = ['hip','knee']
m.J = Set(initialize = joints)
m.fj = Var(m.N, m.J) # net force at each joint

# ground reaction forces
m.GRF = Var(m.N, m.TDOF)

# bound variables
for n in range(1,N+1):
    for l in links:
        m.q[n,'y'].setlb(0.0)
        m.q[n,'th1'].setlb(-0.5*np.pi)
        m.q[n,'th1'].setub(0.5*np.pi)
        m.q[n,'th2'].setlb(-0.5*np.pi)
        m.q[n,'th2'].setub(0.5*np.pi)
        m.q[n,'s'].setlb(0)
        m.q[n,'s'].setub(0.5)
        
    m.fj[n,'hip'].setlb(-2)
    m.fj[n,'hip'].setub(2)
    m.fj[n,'knee'].setlb(-10)
    m.fj[n,'knee'].setub(10)
        
    m.GRF[n,'y'].setlb(0) # GRF can't "pull" towards the ground

## **Contacts**
The problems at the fixed point in the last tutorial started to reveal some of the difficulties around formulating hard boundaries in an optimizaion problem. This tutorial is going to elaborate on those issues and explain the workarounds we use to overcome them. Well, maybe "overcome" is a strong word... often the best we can do is just making the situation less dire so it can actually solve within a human lifetime and only break for half the random seeds we throw at it instead of most of them.

I'll get more specific as we go, but as a general overview, contacts are difficult to deal with for two primary reasons:

**They're discontinuous.** Imagine trying to draw a freebody diagram representing the forces acting on a four-legged animal: you'd have the weight, the torques acting at each of its joints, the normal force on each of its paws... But only if it actually happened to be standing on all four paws. It could have just one foot on the ground, or two, or three, or *the other three*, or none at all or... you get the point. Changing the contact state essentially changes the dynamic model you're working with by switching different forces 'off' or 'on'. 

There is an approach called *hybrid modelling* that makes this explicit: each contact state gets its own dedicated dynamic model with switches between models happening at specified events, such as foot touch-downs and lift-offs. But this isn't suitable for our needs because:
1. The number of discrete models is prone to explode. For a biped, you need four: right foot down, left foot down, neither, both. For a quadruped, you need sixteen. And that's just assuming the foot sticks in place and doesn't slide. 
2. You need to know the contact order up front. Not a problem if you're modelling a well-established motion pattern like a gallop, but our research questions are often around less-defined manoeuvres like rapid stops or turns, or involve finding out the best movement to achieve something, and therefore requires that the model isn't limited to a specific contact order.

...So we need a single model that can apply the right combination of forces under the right conditions. If I had to ask you to describe such a thing using pseudocode or a flow chart, it would probably involve a lot of IF statements:

```python
if right_foot_contact:
    right_GRF_active = True
    ```
...that sort of thing. Problem is, NLP solvers are always (at least, to my knowledge) gradient-based to some extent, and like nice, smooth derivatives they can slide down to an answer. Conditionals like that create sharp cliffs that they absolutely can't deal with. To get around this, we cheat using something called *complementarity*... but we'll get to that just now.

**They're fast.** Which brings me to... 

### **Contacts and timing**
Hard boundaries themselves aren't really the problem - it's actually the dynamics of collisions that's tricky. Hard boundaries merely create a site where collisions must occur.

Collisions are hard because they're *impulsive events:* in continuous time world, their dynamics involve very large forces that act over infinitesimally short time intervals. But in trajectory optimization world, there's a finite minimum timestep over which forces must act. 

Another way to think about it, if you'll allow me a moment to put my Signals and Systems hat on, is that we're attempting to deal with high frequency signals within the confines of a slow sample rate. An IF statement is just a step function, and think of all the high-frequency harmonics you need to make a step look truly *crispy*... Now imagine what it would look like aliased to the 50 - 100 Hz sample rate we typically limit ourselves to. Nyquist is shaking.

So... why not just sample faster? To quote an ancient meme proverb: "Ain't nobody got time for that." Shorter timestep = more nodes to cover the same duration = more variables and constraints = huge problem size = centuries of solving time.

Ideally, what you want is a slow sample rate for the continuous parts and a fast sample rate for when collisions happen. And that's exactly what we do! Within limitations. 

But let's start with the good news: a variable timestep allows the flexible sampling rate we desire, and also allows collisions to occur more naturally. See, the model can only change states at the boundaries of a timestep, meaning that if you're using a fixed step of say 0.01 seconds, the foot can only touch down or the joint hit its limit or whatever at $t=$ some precise multiple of 0.01. Putting some bend in the timestep theoretically allows it to stay in a state for an arbitrary duration.

Unfortunately, there is only so much bend we can allow. Introducing a variable timestep makes the problem even more nonlinear, and we have been warned by Prof Larry Biegler (the First of his Name, High Archmage of trajectory optimization, Coder of IPOPT) that allowing it too much flexibility can cancel out the benefits and make the problem harder to solve. Based on his rule of thumb, we usually only allow it to vary within 20% of a maximum timestep.
___
## **The scaling problem**
If you do any reading on the topic of optimization, you'll probably hear about the *conditioning* of problems. A *well-conditioned* optimization problem is something an NLP solver is well-equipped to deal with: we've already mentioned that solvers like smooth, convex functions. Another thing they really like is a *well-scaled* problem: that is, one where all the variables and constraint derivatives are a similar order of magnitude.

Alas, we find ourselves in an especially difficult predicament here. If you had to describe the worst possible system to do trajectory optimization on... The 9th Circle of Badly-Conditioned Hell... whatever you come up with would probably be pretty close to the systems we work with.

Consider the velocity integration constraint. It includes:
* velocities with magnitudes in 1's or 10's
* accelerations that might have magnitudes similar to the velocities, but can also have magitudes in the 100's, 1000's or even 10 000's when collisions occur
* our now-variable timestep in the order 0.01

In combination with the horribly infeasible starting points our random initialization method creates, it's not surprising that these problems take ages to solve, frequently come back infeasible, or crash the algorithm entirely. Even once your model has passed a bunch of sanity checks and been debugged to the extent that you're confident all the equations, etc. are fundamentally correct, you might still hit the dreaded *restoration failed* error. I don't want to get too in-depth into the technical details of IPOPT, but when you see that, what's happened is the optimization gnome tasked with solving your problem has had a breakdown, given up, flipped his desk and stormed out, viciously cursing you, your robot, your ancestors, and the entire science of mathematics all the while.

### **Scaling - an overview**
One of the things we can do to make our poor gnome's life easier is attempt to scale our problem better. The inherent properties of our system mean its probably never going to be well-scaled, but a bit of tinkering can make it slightly less than a complete trash fire.

One of my few complaints about Pyomo is that it doesn't have an inbuilt scaling feature, meaning we have to introduce scale factors manually. This can be a good thing, since it's much more flexible, and you know exactly how the scaling is implemented, but it does introduce the potential for human error so you need to keep your wits about you.

Scaling is yet another one of those *art* things (I'm sure you're getting really tired of those by now) so I don't really have a precise, step-by-step theory of scaling you can follow. The best I can do is tell you the general guidelines and explain a few specific cases as I go along. 

Here's what we're trying to achieve by scaling:
1. the values of all the variables in the solution should be around 1 ("around 1" here meaning order of magnitude 1 ideally, but somewhere between 0.01 and 100 pragmatically.)
2. The values of all constraint derivatives should be around 1. For linear constraints, this is as simple as making sure all the coefficients are within the established "around 1" range. For nonlinear constraints, it's much trickier, since the "coefficients" (when you think of the partial derivative in a particular variable) become functions of the variables. 
3. The value of each term in a constraint equation should be around 1.

With our problems, it's almost impossible to satisfy all of these at once. You might have to make trade-offs. For variables with wide ranges, work with the mean values you expect them to take on and accept that the nodes where the extreme values occur will be nasty. Most of your variables shouldn't need scaling: start with the most egregious cases (e.g. the timestep or the contact forces, which tend to have magnitudes furthest outside these bounds) and go from there.  

This is all complicated further by the inescapable tangle of interdependency: how well your model works is tightly tied up with the initial guess, the timestep and the duration of the simulation. Finding the right balance of all these things is the process of *conditioning* your model a.k.a. all the kerfuffle that comes after writing and basic sanity checks, but before the model is ready to be rolled out for your experiment. Conditioning is the worst, most rage-inducing part of trajectory optimization, and often takes far longer than actually writing the model. My approach to it is to try to separate each of the aspects into a separate step:
1. **scaling:** I choose whichever scale factors seem logical based on the stated guidelines, adding them iteratively and keeping those that don't appear to actively worsen the performance. Normalization is your friend: you're much more likely to come out with a well-scaled model if you work in units of body mass (i.e. defining the mass of the limb links by the total mass such that the combined mass of the model comes out as 1) than trying to work with measurements in kilograms. Testing the scaling on your target problem can be tricky, since it might not solve at all until you've tweaked the timing, so use a sanity check you know it can pass already. (I provide a few examples at the end of this notebook). The `m.pprint()` readout is super helpful for pinpointing the variables and equations that need the most work.

2. **timing:** Now you're ready to try your target problem. You probably have some idea of how long the motion should take, so start with that. If it doesn't solve, add or remove nodes based on whether $h$ is tending to hit the upper or lower bound. (Technically, you could change the duration by changing the timestep, but I find that causes scaling issues, so I prefer to keep $h_m$ somewhere between 0.01 and 0.02 seconds and adjust the duration using the number of nodes instead.)

3. **initialization:** This is where the model gets most temperamental, so I wouldn't recommend even touching this step until you have a solution converged with the default initial values. Before wasting time here, you need to know that the motion is achievable and how long it takes. Remember: unless you're *warm-starting* from a previous solution, whatever you add here is likely to be less feasible than the default input - if it can't solve from the default point, it definitely can't solve from whatever weird point you inflict on it. 

You could really keep fiddling forever, so it's important to know when to stop: you're not going for *perfect*, or even *good* - you're going for *usable*.

### **Scaling example 1: velocity integration**
We want a master timestep $h_m = 0.02$, meaning the variable timestep has a range $0.016 \leq h[n] \leq 0.02$. We could just create an $h$ variable with those as the bounds, but it fits much better into the Around 1 Zone if we think of the timestep as $h_m h[n]$, since the variable part then has the range $0.8 \leq h[n] \leq 1.0$.

This means that $h_m$ becomes our scaling factor for $h$: wherever we use it in an equation, it will be as the product of the scalar part and the variable part. The integration constraint therefore becomes: $$\dot{x}[n] = \dot{x}[n-1] + 0.02h[n]*\ddot{x}[n]$$

The typical velocity and acceleration values should be around 10, maybe around 100 for the latter, so the partial derivatives and values of individual terms should come out Around 1 ($10 = 10 + 0.02 \times 1 \times 10$). Even if we allow for the more extreme accelerations, the value of that tricky last term should still be on the high side of Around 1 ($0.02 \times 1 \times 10000$). That's probably good enough.

In [ ]:
# variable timestep
hm = 0.02 # master timestep
m.h = Var(m.N, bounds = (0.8,1.0)) # timestep variation

# Integration constraints 
def BwEuler_p(m,n,dof): # Backward Euler integration for positions
    if n > 1:
        return m.q[n,dof] == m.q[n-1,dof] + hm*m.h[n]*m.dq[n,dof]
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to
m.integrate_p = Constraint(m.N, m.DOF, rule = BwEuler_p)

def BwEuler_v(m,n,dof): # Backward Euler integration for velocities
    if n > 1:
        return m.dq[n,dof] == m.dq[n-1,dof] + hm*m.h[n]*m.ddq[n,dof]
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.DOF, rule = BwEuler_v)

# Equations of motion -----------------------------------------------------

def get_var_list(m,n):
    # list the model versions of all quantities in the same order as sym_list
    var_list = [m.g]+\
               [m.m[l] for l in links]+\
               [m.len[l] for l in links]+\
               [m.dCOM[l] for l in links]+\
               [m.In[l] for l in links]+\
               [m.q[n,dof] for dof in DOFs]+\
               [m.dq[n,dof] for dof in DOFs]+\
               [m.ddq[n,dof] for dof in DOFs]+\
               [m.fj[n,j] for j in joints]+\
               [m.GRF[n,dof] for dof in TDOFs]
    return var_list

def dynamics(m,n,dof):
    var_list = get_var_list(m,n)
    return lamb_EOM[dof](*var_list) == 0
m.dynamics = Constraint(m.N, m.DOF, rule = dynamics)

## **Complementarity**
Complementarity is how we finesse conditional behaviour out of functions that are still somewhat compatible with an NLP solver. *Complementing* two quantities just means specifying that their product is zero. This effectively creates a NAND relationship between them: the constraint is satisfied if one or both of the variables is zero, but FALSE if they're both nonzero at the same time. For our purposes, the complemented quantities **must be positive.**

In an ideal world, you'd simply be able to write $AB = 0$ and be done with it, but as you've seen already, trajectory optimization world is far from ideal. Constraints of this form are actually extremely difficult for most NLP solvers to deal with. IPOPT in particular is an interior point solver, which means it looks for the feasible interior region between the multidimensional surfaces formed by the constraints. Complementarity constraints don't have an interior, though: the solution lies along the axis of one complemented variable or the other, since the value of at least one of them must be squashed down to zero. To improve IPOPT's chances, we create a false interior by setting $AB = \varepsilon$, where $\varepsilon$ is a penalty variable we subsequently minimize as part of our cost function. 

Throughout the rest of this tut, you'll see how we construct complementarity constraints to deal with assorted hard contact problems.

## **Ground interactions**

The ground reaction force has two components:
1. the vertical normal force $G_y$. This can only act upwards, so $G_y \geq 0$.
2. the horizontal friction force $G_x$. This can act in either direction, but we represent it using two positive variables $G_x^+ > 0$ and $G_x^- > 0$ such that $G_x = G_x^+ - G_x^-$. Breaking a variable into positive and negative components is something you'll see happening all over the place to facilitate complementarity. Most of the time, it's not necessary to complement these variables with each other, but occassionally you might need to do so to ensure that they can't make weird things happen by being nonzero at the same time. A few back-of-the-envelope calculations never go amiss.

We use four complementarity constraints to bring about this behaviour:

### **Contact**
**Purpose:** Makes sure $G_y$ only acts when the foot is on the ground.

**Constraint:** $G_y[n]y_{foot}[n+1] \leq \varepsilon[n]$

We like complementarity equations to be as simple as possible, so we use an auxiliary variable $y_{foot}$ to represent the foot height. Note that the normal force at *this* timestep is complemented with the foot height at the *next* timestep. Sure this means that the force technically starts acting one node before the foot is directly in contact with the ground, but it is the only way that the nonpenetration condition is actually solveable using an implicit Euler integration.

### **Friction**
**Purpose:** Only allows the foot to have a horizontal velocity (that is, to slide) when it maxes out the static friction limit.

**Constraint:** $(\dot{x}_{foot}^+[n] + \dot{x}_{foot}^-[n])f[n] \leq \varepsilon[n]$

Notice that we've split the foot velocity into positive and negative components, so their sum indicates whether the magnitude is nonzero. 

The aux variable $f$ represents what we call the *friction cone*, though in 2D, it's more like the friction triangle: 

<img src = "friction_cone.png" width = "300">

if the magnitude of the velocity is inside the cone, the foot sticks. If it hits the edge of the cone, the foot slides. Another way of putting it: if the foot is moving while in contact with the ground, the friction force must be at its maximum value $\mu G_y$. The difference between the static friction limit and the friction's actual magnitude is what we define as $f$: $f = \mu G_y - (G_x^+[n] + G_x^-[n])$.

### **Sliding**

**Purpose:** Makes the friction force act in the opposite direction to the foot's velocity.

**Constraint:** This one has two parts - one for each direction: $\dot{x}_{foot}^+[n]G_x^+[n] = p[n]$ and $\dot{x}_{foot}^-[n]G_x^-[n] \leq \varepsilon[n]$

In [ ]:
signs = ['ps','ng']
m.sgn = Set(initialize = signs)

m.footp = Var(m.N, m.TDOF)
def def_footp(m,n,dof):
    var_list = get_var_list(m,n)
    if dof == 'y':
        m.footp[n,dof].setlb(0) # doesn't allow foot below ground level
    return m.footp[n,dof] == lamb_footp[dof](*var_list)
m.def_footp = Constraint(m.N, m.TDOF, rule = def_footp)

m.footv = Var(m.N, m.TDOF, m.sgn, bounds = (0.0,None))
def def_footv(m,n,dof,sgn):
    var_list = get_var_list(m,n)
    return m.footv[n,dof,'ps'] - m.footv[n,dof,'ng'] == lamb_footv[dof](*var_list)
m.def_footv = Constraint(m.N, m.TDOF, m.sgn, rule = def_footv)



m.alpha = Var(m.N, bounds = (0,1))
m.mu = Param(initialize = 0.5)
def friction(m,n):
    if n == 1:
        return Constraint.Skip
    return m.GRF[n,'x'] == m.mu*m.GRF[n,'y']*(1 - 2*m.alpha[n]) # set GRFx = mu * GRFy * term to give it correct sign/direction
m.friction = Constraint(m.N, rule = friction)



gcs = ['contact','sliding_ps','sliding_ng']
m.ground_constraints = Set(initialize = gcs)

m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,None))
def ground_complementarity(m,n,gc):
    # each of these complementarity constraints must be obeyed at all timesteps
    if n == 1:
        return Constraint.Skip
    if gc == 'contact': # this constraint controls the vertical GRF
        A = m.GRF[n-1,'y'] # either { there exists a vertical contact force }
        B = m.footp[n,'y'] + m.footv[n,'y','ps'] + m.footv[n,'y','ng'] # or { the foot is above the ground or moving vertically }

    if gc == 'sliding_ps': # this constraint sets alpha, in order to allow the friction constraint to work
        A = 1 - m.alpha[n] # alpha tends to 1 if there exists a positive foot velocity
        B = m.footv[n,'x','ps']

    if gc == 'sliding_ng': # this constraint sets alpha as well
        A = m.alpha[n] # alpha tends to 0 if there exists a negative foot velocity
        B = m.footv[n,'x','ng']
    
#     return A*B <= 1e-2
    return A*B <= m.ground_penalty[n,gc] # A NAND B. It gets evaluated for each pair above

m.ground_complementarity = Constraint(m.N, m.ground_constraints, rule = ground_complementarity)


# points for plotting
m.plotp = Var(m.N, m.pp, m.TDOF)
def def_plotp(m,n,pp,dof):
    var_list = get_var_list(m,n)
    return m.plotp[n,pp,dof] == lamb_plotp[pp,dof](*var_list)
m.def_plotp = Constraint(m.N, m.pp, m.TDOF, rule = def_plotp)

## **Joints with hard end stops**

You can also use complementarity constraints to create hard boundaries at the end ranges of motion (ROM) for the joints. Constraints on a system's motion are only enforceable if they are literally _enforced_ - as in, if forces or torques are made available to reverse the velocity before they are exceeded. Without hard joint stops, the only forces that can do this will be the actuator forces, which are typically bounded to some approximation of the capabilities of a real-world system. This is not necessarily a problem, but it does remove the possibility of solutions where the joint actuates at full tilt until it hits its limit, as the force would need to reverse direction to keep the joint within the ROM constraints. This can potentially cause oscillatory "bang bang" force profiles to emerge, where the force alternates between its maximum values in opposite directions.

In theory, enforcing joint ROMs as collisions should make a wider range of solutions possible, but personally, I find that the added overhead of extra complementarity constraints negates any positive effect this could have on solver performance. For this reason, I'm not putting them in the Pyomo model, but I'll still show you how to do it just because it might be important to realistically model some system you care about:

### **Hard stops in a rotary joint**
Hard joint stops work exactly like the contact constraint in the ground block, only now, instead of the foot height, it's the distance to the endpoint, and instead of the ground reaction force, it's a rebound action opposing motion beyond the limit.

Only two complementarity constraints are needed - one for each boundary:

On the upper side: $\tau_r^-[n](\theta_{max} - \theta[n+1]) \leq \varepsilon[n]$

And on the lower side: $\tau_r^+[n](\theta[n+1] - \theta_{min}) \leq \varepsilon[n]$

You could also do something like this to set hard limits on the prismatic joint.

## Minimizing the penalty
There are three ways to approach minimizing the complementarity penalties:
1. Simultaneous: add them as a term to the cost function, typically magnified by some large scale factor so flattening them is prioritized. I normally go with 1000, or whatever amounts to around three orders of magnitude greater than whatever I'm trying to minimize.
2. Iterative: here, the complementarity penalties aren't added to the cost function. Rather, you give them an upper bound and then solve the problem repeatedly, decreasing this bound each time.
3. Two stage: first, only the sum of the penalties is minimized to get a feasible solution. The penalties are then bounded to below an acceptable value (say, $1e-4$) and the problem is then solved again with the intended objective being minimized.

Personally, I find 3 to be the most reliable, so that's what I use in my research. I'll be using 1 here, because it's the simplest and the only option that completes in a single solve stage, but be warned that scaling issues can arise due to the large scale factor on the penalties. It often takes a bit of tweaking to find a suitable weight that results in sufficiently small penalties but doesn't break the whole problem.

In [ ]:
# COST FUNCTION -------------------------------------------------------------------------------------------------------------

# minimum time

def CostFun(m):
    T = sum(m.h[n] for n in range(1,N+1))
    penalty_sum = sum([m.ground_penalty[n,gc] for n in range(1,N+1) for gc in m.ground_constraints])
    # ain't single-line for loops grand?
    return T+1000*penalty_sum

#     return penalty_sum # for sanity checks
m.Cost = Objective(rule = CostFun)

## Sanity checks
One of the tricky and often frustrating things about trajectory optimization is that you have to code everything before you can test anything. Due to the aforementioned interdependency issue, it can be very difficult to pinpoint the exact cause of strange behaviour or isolate any one part to check if it's working correctly.

To help with this, there are a few simple tests I like to run right after finishing a model to help find and kill some of the more obvious bugs. Because the cost function can potentially lead the solver into a difficult place, I'd recommend running these with a *feasibility* objective: that is, no other goal besides satisfying the penalty. 

### The high drop test
**How you do it:** Set the initial condition to rest in a known pose (say, all angles = 0) high enough above the ground that it can fall without hitting it, fix all actuator forces/ torques to zero and then just let it drop.

**What's the point?** This test will reveal a few things:
1. Big, stupid problems: if it comes back infeasible, your model is broken somewhere. Time to make yourself some coffee, dust off the ol' `m.pprint()` statement and try to find that one equation you copied and forgot to change a variable name in.
2. Ground interaction problems: if a ground reaction force is anything other than zero when it's three metres in the air, something's wrong.
3. Equations of motion problems: check `m.ddq.pprint()`. You want to see zero acceleration everywhere, and -9.81 in the $y$ direction. If you see anything else, something's wrong. This is especially important when you're working in system space coordinates, where you have all those tricky constraint forces to balance; you have to make sure they're not somehow conspiring to create motion where none should happen (this is why we set the actuator actions to zero).

In [ ]:
# # HIGH DROP -----------------------------------------------------------------------------------------------------------------

# # initial condition
# for dof in DOFs:
#     m.dq[1,dof].fix(0) # rest
#     m.q[1,dof].fix(0)  # neutral posture
    
# m.q[1,'y'].unfix() # we don't want to start at ground level
# m.footp[1,'y'].fix(10)

# # during
# for n in range(1,N+1):
#     for j in joints:
#         m.fj[n,j].fix(0) # no forces

### The low drop test
**How you do it:** Same as the high drop, but now you're starting with the foot only a short way above the ground (e.g. 1 cm) and fixing the foot to be grounded at some time, so it's guaranteed to land.

**What's the point?** To check if your ground contacts work to activate the ground reaction forces and successfully stop the feet.

In [ ]:
# # LOW DROP -----------------------------------------------------------------------------------------------------------------

# # initial condition
# for dof in DOFs:
#     m.dq[1,dof].fix(0) # rest
#     if dof not in ['y','s']:
#         m.q[1,dof].fix(0) # neutral posture
# m.footp[1,'y'].setlb(0.1) # start near the ground

# m.footp[15,'y'].fix(0) # force it to land
# for n in range(1,N+1):
#     for j in joints:
#         m.fj[n,j].fix(0) # no forces

### The standing test
**How you do it:** Deactivate the actuators as in the other tests, and start the model at rest in a known pose with its feet (well, foot in this case) on the ground. It might also be worth trying a few postures that push the limits of the joints e.g. have the prismatic knee joint compressed all the way in.

**What's the point?** While the low drop is meant to check if your contacts behave as expected in impulsive collisions, the standing test checks whether they can handle sustained contact.

In [ ]:
# # STANDING ------------------------------------------------------------------------------------------------------------------

# # initial condition
# for dof in DOFs:
#     m.dq[1,dof].fix(0) # rest
#     if dof not in ['y','s']:
#         m.q[1,dof].fix(0) # neutral posture

# for n in range(1,N+1):
#     m.footp[n,'y'].fix(0) # must stay grounded for the full time

# for n in range(1,N+1):
#     for j in joints:
#         if j != 'knee': # needs knee force to maintain ROM limits of prismatic joint
#             m.fj[n,j].fix(0) # no forces

### The hop test
**How you do it:** Now we get to add some power :) Start the model at rest in contact with the ground, and set the final condition to some $x$ or $y$ that will either force it to move forward or jump in the air. 

**What's the point?** To see if the actuator force limits you've set allow enough *oomph* to get the dude off the ground. If you're brave and want to test the limits in the other direction, you can tell it to maximize the final $x$ or $y$ position (by making $-x[N]$ or $-y[N]$ your objective) to make sure it can't shoot itself into orbit, either.

**Note:** To save on time, use as few nodes as possible to do these tests. Especially for something like the high drop, you should be able to see what you're trying to see with just $N = 10$.

In [ ]:
# # HOP -----------------------------------------------------------------------------------------------------------------------
# # initial condition
# for dof in DOFs:
#     m.dq[1,dof].fix(0) # rest
#     if dof not in ['y','s']:
#         m.q[1,dof].fix(0) # neutral posture
# m.footp[1,'y'].fix(0)

# # midpoint
# m.footp[10,'y'].setlb(0.2)

# # final condition
# for dof in DOFs:
#     m.dq[N,dof].fix(0) # rest
#     if dof not in ['y','s']:
#         m.q[N,dof].fix(0) # neutral posture
# m.footp[N,'y'].fix(0)

In [ ]:
#SPRINT --------------------------------------------------------------------------------------------------------------------
#sprint 5m from rest
# initial condition
for dof in DOFs:
    m.dq[1,dof].fix(0) # rest
    if dof not in ['y','s']:
        m.q[1,dof].fix(0) # neutral posture
m.footp[1,'y'].fix(0)

# final condition
m.q[N,'x'].fix(5)
m.footp[N,'x'].fix(5)

In [ ]:
# solving
opt = SolverFactory('ipopt') # standard issue, garden variety ipopt

# opt = SolverFactory('ipopt',executable = 'C:/cygwin64/home/Stacey/CoinIpopt/build/bin/ipopt.exe')
# opt.options["linear_solver"] = 'ma97'

# solver options
opt.options["expect_infeasible_problem"] = 'yes'
#pt.options["linear_system_scaling"] = 'none'
#opt.options["mu_strategy"] = "adaptive"
opt.options["print_level"] = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"] = 30000 # maximum number of iterations
opt.options["max_cpu_time"] = 600 # maximum cpu time in seconds
opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
results = opt.solve(m, tee = True)

In [ ]:
# if the problem is infeasible, this is how you can see which constraints weren't satisfied
from pyomo.util.infeasible import log_infeasible_constraints
log_infeasible_constraints(m)

In [ ]:
print(results.solver.status) # tells you if the solver had any errors/ warnings
print(results.solver.termination_condition) # tells you if the solution was (locally) optimal, feasible, or neither.

penalty_sum = 0
for n in range(1,N+1):
    for gc in m.ground_constraints:
        penalty_sum += m.ground_penalty[n,gc].value
print(penalty_sum)

#m.pprint() 

In [ ]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1) #create axes
ax1.set_aspect('equal')

xmin = np.min([m.plotp[n,pp,'x'].value for n in range(1,N+1) for pp in m.pp])
xmax = np.max([m.plotp[n,pp,'x'].value for n in range(1,N+1) for pp in m.pp])
ymax = np.max([m.plotp[n,pp,'y'].value for n in range(1,N+1) for pp in m.pp])

def plot_pendulum(i,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([xmin-1, xmax+1])
    ax.set_ylim([     0, ymax+1])
    
    # plot link 1
    BodyLx = m.plotp[i,'bodyL','x'].value
    BodyLy = m.plotp[i,'bodyL','y'].value
    BodyRx = m.plotp[i,'bodyR','x'].value
    BodyRy = m.plotp[i,'bodyR','y'].value
    ax.plot([BodyLx,BodyRx],[BodyLy,BodyRy],color='xkcd:green')
    
    
    # plot link 2
    Hipx = m.plotp[i,'hip','x'].value
    Hipy = m.plotp[i,'hip','y'].value
    Knee1x = m.plotp[i,'knee1','x'].value
    Knee1y = m.plotp[i,'knee1','y'].value
    ax.plot([Hipx,Knee1x],[Hipy,Knee1y],color='xkcd:black')
    
    # plot prismatic joint
    Knee2x = m.plotp[i,'knee2','x'].value
    Knee2y = m.plotp[i,'knee2','y'].value
    ax.plot([Knee1x,Knee2x],[Knee1y,Knee2y],color='xkcd:red')
    
    # plot link 3
    Footx = m.plotp[i,'foot','x'].value
    Footy = m.plotp[i,'foot','y'].value
    ax.plot([Knee2x,Footx],[Knee2y,Footy],color='xkcd:blue')
    
update = lambda i: plot_pendulum(i,m,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True) # interval = frame time. 1/50 = 20 fps
# animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 1000*hm,repeat=True) # if you want it to play at the actual speed

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook
# ani.Animation.save(animate,'placeholder.mp4', fps=int(1/h), dpi=300) # if you want to save the animation instead of embedding it